In [ ]:
from tensorflow.keras.datasets import cifar100
import tensorflow.keras as keras
import  tensorflow as tf
from    tensorflow import keras
from    tensorflow.keras import layers, models, Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization, Activation, GlobalAveragePooling2D,Dropout
from tensorflow.python.keras.callbacks import LearningRateScheduler,EarlyStopping
# (x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')
(x_train,y_train),(x_test,y_test)=cifar100.load_data(label_mode="fine")
y_train=keras.utils.to_categorical(y_train,100)
y_test=keras.utils.to_categorical(y_test,100)

x_train=x_train.astype('float32')
x_test=x_test.astype('float32')

x_train/=255.0
x_test/=255.0


learning_rate=0.1
def scheduler(epoch):
    if epoch<=60:
      learning_rate=0.1
    else:
      learning_rate=0.02
    return learning_rate
change_Lr = keras.callbacks.LearningRateScheduler(scheduler)
# early_stopping = EarlyStopping(monitor='acc', patience=5, verbose=1)
early_stopping=keras.callbacks.EarlyStopping(monitor="acc",patience=5,verbose=1)
#定义shortcut+卷积类
class conv_block(keras.Model):
  def __init__(self,filter_num,stride,Name):
    self.Name=Name
    super(conv_block,self).__init__()
    
    self.conv1=Conv2D(filter_num[1],kernel_size=3,strides=stride,kernel_initializer=keras.initializers.variance_scaling(),padding="same",activation="relu",name=self.Name+"conv1")
    self.conv2=Conv2D(filter_num[1],kernel_size=3,strides=stride,kernel_initializer=keras.initializers.variance_scaling(),padding="same",activation="relu",name=self.Name+"conv2")
    self.shortcut=Conv2D(filter_num[2],kernel_size=3,strides=stride,kernel_initializer=keras.initializers.variance_scaling(),padding="same",name=self.Name+"shortcut")
    self.norm=BatchNormalization(name=Name+"norm")
    self.act=Activation("relu",name=self.Name+"act1")
  def call(self,inputs):
    # inputs=keras.layers.Reshape(target_shape=(32,32,3))(inputs)
    x=self.conv1(inputs)
    x=self.norm(x)
    x=self.conv2(x)
    x=self.norm(x)
    shortcut=self.shortcut(inputs)
    x=keras.layers.add([x,shortcut])
    x=self.act(x)
    return x
#定义非shortcut++卷积类
class identity_blocks(keras.Model):
  def __init__(self,filter_num,stride,Name):
    self.Name=Name
    super(identity_blocks,self).__init__()
    self.conv1=Conv2D(filter_num[1],kernel_size=3,strides=stride,kernel_initializer=keras.initializers.variance_scaling(),padding="same",activation="relu",name=self.Name+"conv1")
    self.conv2=Conv2D(filter_num[1],kernel_size=3,strides=stride,kernel_initializer=keras.initializers.variance_scaling(),padding="same",activation="relu",name=self.Name+"conv2")
    self.norm=BatchNormalization(name=Name+"norm")
    self.act=Activation("relu",name=Name+"relu")
  
  def call(self,inputs):
    
    x=self.conv1(inputs)
    x=self.norm(x)
    x=self.conv2(x)
    x=self.norm(x)
    x=keras.layers.add([x,inputs])
    x=self.act(x)
    return x

class ResNet18(keras.Model):
  def __init__(self,Name):
    self.Name=Name
    super(ResNet18,self).__init__()
    self.conv=Conv2D(64,kernel_size=7,strides=2,kernel_initializer=keras.initializers.variance_scaling(),padding="same",name=self.Name+"conv_1",activation="relu")
    self.norm=BatchNormalization(name=self.Name+"norm1")
    self.pool=MaxPooling2D(pool_size=3,strides=2,padding="same")
    self.identity_1=identity_blocks(filter_num=[48,64,64],stride=1,Name=self.Name+"identity_1")
    self.identity_2=identity_blocks(filter_num=[48,64,64],stride=1,Name=self.Name+"identity_2")
    self.conv_block1=conv_block(filter_num=[96,128,128],stride=1,Name=self.Name+"conv_1")
    self.identity_3=identity_blocks(filter_num=[96,128,128],stride=1,Name=self.Name+"identity_3")
    self.conv_block2=conv_block(filter_num=[128,256,256],stride=1,Name=self.Name+"conv_2")
    self.identity_4=identity_blocks(filter_num=[128,256,256],stride=1,Name=self.Name+"identity_4")
    self.conv_block3=conv_block(filter_num=[256,512,512],stride=1,Name=self.Name+"conv_3")
    self.identity_5=identity_blocks(filter_num=[256,512,512],stride=1,Name=self.Name+"identity_5")
    self.avg_pool=GlobalAveragePooling2D()
    self.d=Dense(100,activation="softmax",name=self.Name+"output")
    
    
  def call(self,inputs):
    # Inputs=keras.layers.Reshape(target_shape=(32,32,3))(inputs)
    x=self.conv(inputs)
    x=self.norm(x)
    x=self.pool(x)
    x=self.identity_1(x)
    x=self.identity_2(x)
    x=Dropout(0.4)(x)
    x=self.conv_block1(x)
    x=self.identity_3(x)
    x=Dropout(0.25)(x)
    x=self.conv_block2(x)
    x=self.identity_4(x)
    x=Dropout(0.25)(x)
    x=self.conv_block3(x)
    x=self.identity_5(x)
    x=Dropout(0.25)(x)
    x=self.avg_pool(x)
    output=self.d(x)
    return output

Model=ResNet18("ResNet18")
# Model=conv_block(filter_num=[48,64,64],stride=1,Name="conv")

Model.build(input_shape=(None,32,32,3))
Model.summary()
Adam=keras.optimizers.Adam(learning_rate=learning_rate)
Model.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["acc"])
history=Model.fit(x_train,y_train,epochs=100,batch_size=32,validation_data=(x_test,y_test))

